In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/datasets/predictive_maintenance.csv'
data = pd.read_csv(path)

###### Q1 : Data Pre-processing

X = data.iloc[:, 3:8].values
y = LabelEncoder().fit_transform(data['Target'])

# Standardization/Normalization
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
###### Q2 : Exploratory Data Analysis (EDA)

# Understand the structure of the data
print(data.describe())
print(data.info())
print(data.isnull().sum())

# Visualize the data
sns.pairplot(data)
plt.show()

In [ ]:
###### Q3 : Data augmentation technics

minority_class = 'Failure'

minority_data = data[data['Target'] == minority_class]
majority_data = data[data['Target'] != minority_class]

augmented_data = resample(minority_data, replace=True, n_samples=len(majority_data), random_state=42)
augmented_dataset = pd.concat([majority_data, augmented_data])
augmented_dataset = augmented_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
###### Q4 : Deep Neural network Architecture

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Deep Neural Network Architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Model Initialization
input_size = X_train.shape[1]
output_size = len(np.unique(y))
hidden_size = 64
model = NeuralNetwork(input_size, hidden_size, output_size)

In [ ]:
###### Q5 : GridSearch for Hyperparameters

param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'module__hidden_size': [32, 64, 128],
}

scoring = {'accuracy': make_scorer(accuracy_score)}

grid_search = GridSearchCV(net, param_grid, scoring=scoring, refit='accuracy', cv=3)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

y_pred_test = best_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Test Accuracy (Best Model):", accuracy_test)

In [ ]:
###### Q6 : Visualization


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 50
train_losses, test_losses, accuracies = [], [], []

for epoch in range(epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.FloatTensor(X_train))
    loss = criterion(outputs, torch.LongTensor(y_train))
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Testing
    model.eval()
    with torch.no_grad():
        test_outputs = model(torch.FloatTensor(X_test))
        test_loss = criterion(test_outputs, torch.LongTensor(y_test))
        test_losses.append(test_loss.item())

        _, predicted = torch.max(test_outputs.data, 1)
        accuracy = accuracy_score(y_test, predicted.numpy())
        accuracies.append(accuracy)

# Visualization
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

plt.plot(accuracies, label='Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
###### Q6 : Metrics Calculation

model.eval()
with torch.no_grad():
    train_outputs = model(torch.FloatTensor(X_train))
    _, train_predicted = torch.max(train_outputs.data, 1)
    test_outputs = model(torch.FloatTensor(X_test))
    _, test_predicted = torch.max(test_outputs.data, 1)

print("Training Accuracy:", accuracy_score(y_train, train_predicted.numpy()))
print("Test Accuracy:", accuracy_score(y_test, test_predicted.numpy()))
print("\nClassification Report (Test Data):\n", classification_report(y_test, test_predicted.numpy()))
print("\nConfusion Matrix (Test Data):\n", confusion_matrix(y_test, test_predicted.numpy()))
